In [ ]:
import os
import re
import json
import fitz
import spacy
import faiss
import pickle
import win32com.client
from docx import Document
import ipywidgets as widgets
from IPython.display import display, Javascript
from RAGLibrary import Widgets

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
widgets_list = Widgets.create_name_form()

In [ ]:
""" DEFINE """

""" GET PARENT VALUE """
data   = widgets_list[0] #HBox 1
keys   = widgets_list[1] #HBox 2
choose = widgets_list[2] #HBox 3

embedd_model = widgets_list[3]
search_egine = widgets_list[4]
rerank_model = widgets_list[5]
respon_model = widgets_list[6]
API_drop     = widgets_list[7]

# Hbox 4
chunk_input = widgets_list[8]

# HBox 5
level_value = widgets_list[9]

# Button
button_box  = widgets_list[10]


""" GET CHILDREN VALUE """

# HBox 1
file_name = data.children[0]
file_type = data.children[1]
path_end  = data.children[2]

# HBox 2
data_key = keys.children[0]
embe_key = keys.children[1]

# HBox 3
switch_model = choose.children[0]
merge_otp    = choose.children[1]

# HBox 4
level_input = chunk_input.children[0]
word_limit  = chunk_input.children[1]

# HBox 5
level_values = [child.value for child in level_value.children]

""" DEF VALUE """

# Get value
data_folder   = file_name.value
file_type_val = file_type.value

data_key_val  = data_key.value
embe_key_val  = embe_key.value

API_key_val = API_drop.value
switch      = switch_model.value
merge       = merge_otp.value
path_end_v  = path_end.value

embedding_model = embedd_model.value
searching_egine = search_egine.value
reranking_model = rerank_model.value
responing_model = respon_model.value

level_input_val = level_input.value
word_limit_val  = word_limit.value


""" DETAIL """

# Define
dcmt_path = f"../Doc/{data_folder}{path_end_v}"

base_folder = "../Data"
base_path = f"{base_folder}/{data_folder}/{file_type_val}_{data_folder}"

chunks_base    = f"{base_path}_Chunks.json"
json_file_path = f"{base_path}_Database.json"
schema_ex_path = f"{base_path}_Schema.json"
embedding_path = f"{base_path}_Embeds_{merge}"

torch_path  = f"{embedding_path}.pt"
faiss_path  = f"{embedding_path}.faiss"
mapping_path = f"{embedding_path}_mapping.json"
mapping_data = f"{embedding_path}_map_data.json"

FILE_TYPE    = file_type_val
DATA_KEY     = data_key_val
EMBE_KEY     = embe_key_val
SWITCH       = switch
EMBEDD_MODEL = embedding_model
SEARCH_EGINE = searching_egine
RERANK_MODEL = reranking_model
RESPON_MODEL = responing_model

if FILE_TYPE == "Data":
    MERGE = merge
else: 
    MERGE = "no_Merge"

API_KEY = API_key_val

SEARCH_ENGINE = faiss.IndexFlatIP

print("\n")
print(f"Embedder: {EMBEDD_MODEL}")
print(f"Searcher: {SEARCH_EGINE}")
print(f"Reranker: {RERANK_MODEL}")
print(f"Responer: {RESPON_MODEL}")
print(f"Data Key: {DATA_KEY}")
print(f"Embe Key: {EMBE_KEY}")
print(f"Dcment  : {dcmt_path}")
print(f"Chunked : {chunk_base}")
print(f"Database: {json_file_path}")
print(f"Torch   : {torch_path}")
print(f"Faiss   : {faiss_path}")
print(f"Mapping : {mapping_path}")
print(f"Map Data: {mapping_data}")
print(f"Schema  : {schema_ex_path}")
print(f"Model   : {SWITCH}")
print(f"Merge   : {MERGE}")
print(f"API Key : {API_KEY}")
print(f"Word    : {word_limit_val}")
print(f"Level   : {level_input_val}")
print(f"Level Values: {level_values}")

In [ ]:
path = dcmt_path

BASE


In [ ]:
def sentence_end(text):
    brackets = ["()", "''", '""', "[]", "{}", "«»", "“”", "‘’"]
    valid_brackets = any(text.startswith(pair[0]) and text.endswith(pair[1]) for pair in brackets)
    valid_end = text.endswith(('.', '!', '?', ':', ';'))
    return valid_end or valid_brackets

def markers(text):
    return bool(re.match(r'^([-+*•●◦○] )|([0-9a-zA-Z\-\+\*ivxIVX]+[.)\]:] )|(\(\d+\) )|(\(\w+\) )|([0-9]+\s+-\s+[0-9]+ )', text))

def unclosed(text):
    stack = []
    brackets = {"(": ")", "[": "]", "{": "}", '"': '"', "'": "'", "«": "»", "“": "”", "‘": "’"}
    for char in text:
        if char in brackets.keys():
            stack.append(char)
        elif char in brackets.values():
            if stack and brackets[stack[-1]] == char:
                stack.pop()
            else:
                return False
    return bool(stack)

def merge_text(para, new_para):
    should_merge = (not (new_para.isupper() ^ para.isupper()) and not markers(new_para) and (not new_para[0].isupper() or not sentence_end(para))) or unclosed(para)
    return should_merge

def extracted(path):
    file_ext = os.path.splitext(path)[1].lower()
    text_data = []

    if file_ext == ".docx":
        doc = Document(path)
        paragraph = ""
        for para in doc.paragraphs:
            for line in para.text.split("\n"):
                cleaned_text = ' '.join(line.strip().split())
                if cleaned_text:
                    if paragraph and merge_text(paragraph, cleaned_text):
                        paragraph += " " + cleaned_text
                    else:
                        if paragraph:
                            text_data.append({"text": paragraph})
                        paragraph = cleaned_text
        if paragraph:
            text_data.append({"text": paragraph})

    elif file_ext == ".doc":
        word = win32com.client.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(os.path.abspath(path))
        text = doc.Content.Text
        doc.Close()
        word.Quit()

        paragraph = ""
        for line in text.split("\n"):
            cleaned_text = ' '.join(line.strip().split())
            if cleaned_text:
                if paragraph and merge_text(paragraph, cleaned_text):
                    paragraph += " " + cleaned_text
                else:
                    if paragraph:
                        text_data.append({"text": paragraph})
                    paragraph = cleaned_text
        if paragraph:
            text_data.append({"text": paragraph})

    elif file_ext == ".pdf":
        doc = fitz.open(path)
        paragraph = ""
        for page in doc:
            blocks = sorted(page.get_text("blocks"), key=lambda b: (b[1], b[0]))
            for block in blocks:
                for line in block[4].split("\n"):
                    cleaned_text = " ".join(line.strip().split())
                    if cleaned_text:
                        if paragraph and merge_text(paragraph, cleaned_text):
                            paragraph += " " + cleaned_text
                        else:
                            if paragraph:
                                text_data.append({"text": paragraph})
                            paragraph = cleaned_text
        if paragraph:
            text_data.append({"text": paragraph})

    return text_data

In [ ]:
# ADD CHUNKS
def add_chunk(chunks, content):
    if content["Chương"] and content["Nội dung"]:
        content["Index"] += 1
        chunks.append(content.copy())
        content["Nội dung"] = []
        
def is_chapter(text):
    text = text.strip()
    return bool(re.match(r"^Chương\s*[IVXLCDM\d]+\b", text, re.IGNORECASE))

def is_article(text):
    text = text.strip()
    return bool(re.match(r"^Điều\s+([IVXLCDM\d]+)\.\s*(.+)", text, re.IGNORECASE))

def is_clause(text):
    text = text.strip()
    return bool(re.match(r"^\d+\.\s+.+", text))

def is_content(text):
    text = text.strip()
    return bool(re.match(r'^([-+*•●◦○] )|([a-zA-Z\-\+\*]+[.)\]:] )|(\(\w+\) )', text))


In [ ]:
# MAIN FUNCTION
def main(text_data):
    chunks = []
    content = {"Index": 0, "Chương": None, "Điều": None, "Khoản": None, "Nội dung": []}
    i = 0
    while i < len(text_data):
        chunk = text_data[i]["text"]

        if is_chapter(chunk):
            if i + 1 < len(text_data):
                chunk += f": {text_data[i + 1]['text']}"
            add_chunk(chunks, content)
            content["Chương"] = chunk
            content["Điều"] = None
            content["Khoản"] = None 
            i += 1

        elif is_article(chunk):
            match = re.match(r"^(Điều\s*[IVXLCDM\d]+)\.\s*(.+)", chunk, re.IGNORECASE)
            if content["Chương"]:
                if match:
                    chunk = f"{match.group(1)}: {match.group(2)}"
                add_chunk(chunks, content)
                content["Điều"] = chunk
                content["Khoản"] = None 
        
        elif is_clause(chunk):
            match = re.match(r"^(\d+)\.\s*(.+)", chunk)
            if content["Chương"]:
                if match:
                    clause_number = match.group(1)
                    clause_content = match.group(2)

                    if i + 1 < len(text_data) and is_content(text_data[i + 1]["text"]):
                        chunk = f"Khoản {clause_number}: {clause_content}"
                        add_chunk(chunks, content)
                        content["Khoản"] = chunk
                    else:
                        chunk = f"Khoản {clause_number}"
                        add_chunk(chunks, content)
                        content["Khoản"] = chunk

                        chunk = clause_content
                        content["Nội dung"].append(chunk)
                else: 
                    print(chunk)

        elif is_content(chunk):
            match = re.match(r'^([-+*•●◦○a-zA-Z\-\+\*]+[.)\]:] )(\s.+)', chunk)
            if content["Chương"]:
                if match:
                    chunk = match.group(2)
                content["Nội dung"].append(chunk)
        i += 1
        
    return chunks

In [ ]:
# CHUNKS BASE
text_data = extracted(dcmt_path)
chunks = main(text_data)
with open("extract_text.json", "w", encoding="utf-8") as f:
    json.dump(text_data, f, indent=4, ensure_ascii=False)

In [ ]:
# EXPORT BASE
with open(chunks_base, "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=4, ensure_ascii=False)

print(f"Base data saved to {chunks_base}")

FINAL


In [ ]:
nlp = spacy.load("en_core_web_sm")
WORD_LIMIT = int(word_input.value) if word_input.value.isdigit() else 200

In [ ]:
# COUNT WORDS
def count_words(text):
    return len(text.split())

In [ ]:
# CHUNKING IF WORD LIMIT EXCEEDED
def semantic_chunking(text, max_words=WORD_LIMIT):

    doc = nlp(text)
    chunks, current_chunk = [], []
    word_count = 0
    
    for sent in doc.sents:
        sentence = sent.text.strip()
        sentence_length = len(sentence.split())
        
        if word_count + sentence_length > max_words and current_chunk:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            word_count = 0
            
        current_chunk.append(sentence)
        word_count += sentence_length
        
    if current_chunk:
        chunks.append(" ".join(current_chunk))
        
    return chunks

In [ ]:
# MAIN PROCESSING FUNCTION
def process_json(chunks_base, chunks_final):
    with open(chunks_base, "r", encoding="utf-8") as f:
        data = json.load(f)
        
    processed_data = []
    
    for idx, chunk in enumerate(data):
        
        # PRINT PROGRESS INFO
        if "Nội dung" in chunk and isinstance(chunk["Nội dung"], list):
            new_content = []
            
            for para_idx, paragraph in enumerate(chunk["Nội dung"]):
                word_count = count_words(paragraph)
                
                if word_count > WORD_LIMIT  and not markers(paragraph):
                    
                    # WORDS NUMBER                    
                    chunked_paragraphs = semantic_chunking(paragraph)
                    new_content.extend(chunked_paragraphs)
                    
                    # PRINT SEGMENTS NUMBER
                    print(f"{idx+1:04} / {len(data):04}: {len(chunked_paragraphs):02} segments.")
                    
                else:
                    new_content.append(paragraph)
                    
            chunk["Nội dung"] = new_content
            
        processed_data.append(chunk)
        
        # SAVE PROGRESS
        with open(chunks_final, "w", encoding="utf-8") as f:
            json.dump(processed_data, f, indent=4, ensure_ascii=False)
        print(f"{idx+1:04} / {len(data):04}: Saved!\n")
    
    # FINISHED    
    print(f"Final data saved to {chunks_final}.")

In [ ]:
process_json(chunks_base, chunks_final)